# poplib
- poplib는 pop3 서버에 연결하여 받은 이메일을 확인하는데 사용하는 모듈
- 매번 gmail과 같은 웹 메일에 접속해서 새로운 메일 내용 확인이 번거로울 때 이를 자동화할 경우 사용

In [67]:
import poplib

# 이메일 계정 연결

In [68]:
# 암호화된 소켓 연결 (기본 995 포트)
server = poplib.POP3_SSL('outlook.office365.com, port=995')

gaierror: [Errno 11001] getaddrinfo failed

In [69]:
# id 입력
server.user('aaa@aaa.com')

error_proto: b'-ERR Command not accepted'

In [ ]:
# password 입력
import getpass

server.pass_(getpass.getpass())

> #### Gmail 계정을 사용할 경우
>  - 2022년 5월 30일부터 Google은 **사용자 이름**과 **비밀번호**만 사용하여 계정에 로그인하도록 요청하는 서드 파티 앱 또는 기기의 사용을 더 이상 지원하지 않습니다. ([공지문](https://support.google.com/accounts/answer/6010255))
>  - 연동 방법
>     - ① 2단계 인증 활성화 ([설정절차](https://support.google.com/accounts/answer/185839?hl=ko&co=GENIE.Platform%3DAndroid))
>     - ② `Google 계정 관리` > `보안` > `앱 비밀번호` > `메일` 앱 선택 > `기기` (Windows 컴퓨터) > `생성`
>     - ③ 생성된 16자리 비밀번호 사용

In [74]:
# 암호화된 소켓 연결 (기본 995 포트)
server = poplib.POP3_SSL('pop.googlemail.com', port=995)

# google id 입력
server.user('polozhzh')

b'+OK send PASS'

In [75]:
# password 입력
import getpass

server.pass_(getpass.getpass())

b'+OK Welcome.'

In [76]:
# pop3 에 저장된 메일 개수, 크기(bytes)
server.stat()

(2, 143460)

In [77]:
# 가장 최근 메시지 번호
recent_no = server.stat()[0]
print(recent_no)

2


In [78]:
# 메시지 가져오기 (바이트 문자열, 두번째 요소가 메시지 내용)
server.retr(recent_no)

(b'+OK message follows',
 [b'Delivered-To: polozhzh@gmail.com',
  b'Received: by 2002:a05:612c:2c0d:b0:431:bd56:f9ea with SMTP id ir13csp552624vqb;',
  b'        Tue, 21 Nov 2023 04:04:36 -0800 (PST)',
  b'X-Google-Smtp-Source: AGHT+IE5pYCCpK+xKb7TNc+vu0wj3T6RXMeF/bzjbV2fvaejYIErgoRwThD7Em4vzIynxoJUTg4v',
  b'X-Received: by 2002:a0d:f845:0:b0:5af:f026:a27 with SMTP id i66-20020a0df845000000b005aff0260a27mr6818709ywf.50.1700568276001;',
  b'        Tue, 21 Nov 2023 04:04:36 -0800 (PST)',
  b'ARC-Seal: i=1; a=rsa-sha256; t=1700568275; cv=none;',
  b'        d=google.com; s=arc-20160816;',
  b'        b=OrHnr/jVKX/eeMayWVmbD0hNCfTS8kj883Z8B6dKyCgKrQaWwBfOMpenldeMknKVfV',
  b'         GdNG7JEYrA+eZhZ4Ha7VdKQD4lrSzkQyO4eBFbo8n5SWMbwveVSAl4Z9lGobcBixxpwl',
  b'         v2W9RelL7fafYP+sDc5gecBZJrzgrB+mgt784DG4Um3A2K4AgWDO4HAzoDVvy4uONlBX',
  b'         nGlqktRhcIpT4QdS8O7ddQrk6qlz1NZT0l0pbGIQNjfomu3W0gpY7+oMD3rSxdNwXlFR',
  b'         LJaf83JD1+mKswEIIZ2gQiPCVb+tdyTzLOLhsbf8242cS7J641muoRR3fb

In [ ]:
# 줄바꿈으로 메시지 내용 조인
raw_email = b'\n'.join(server.retr(recent_no)[1])

In [ ]:
raw_email

# email
- email 은 바이트 문자열 형식의 이메일 메시지를 사람이 알아볼 수 있는 문자열 형태로 파싱 및 디코딩할 때 사용되는 모듈

In [70]:
import email
from email.header import decode_header, make_header

In [71]:
# 메시지 객체 생성
message = email.message_from_bytes(raw_email)

In [72]:
message

# 이메일 정보 확인

In [73]:
# 송신자 확인
message.get('From')

'=?utf-8?Q?Fidan=20Alizada?= <fidan.alizada@linkgraph.io>'

In [ ]:
# email 모듈 디코더 활용
fr = make_header(decode_header(message.get('From')))
print(fr)

In [ ]:
# 제목 확인
subject = make_header(decode_header(message.get('Subject')))
print(subject)

# 이메일 본문 확인
이메일의 메시지는 멀티파트(multipart)로 구성될 수도 있고 아닐 수도 있음
이메일에 첨부 파일이 있거나 text/plain 또는 text/html 형식의 내용을 동시에 보낸다면 멀티파트 형식으로 구성됨
멀티파트인지 아닌지 분기해서 확인

In [80]:
if message.is_multipart():
    # 멀티 파트라면 여러 개로 나누어진 메시지를 하나씩 처리
    for part in message.walk():
        ctype = part.get_content_type()
        cdispo = str(part.get('Content-Disposition'))

        # 컨텐츠가 text/plain 이고, 첨부파일이 없다면
        if ctype == 'text/plain' and 'attachment' not in cdispo:
            body = part.get_payload(decode=True)  # 메시지 내용 추출
            break
else:  # 싱글 파트라면
    body = message.get_payload(decode=True)  # 메시지 내용 추출

In [81]:
body = body.decode('utf-8')

print(f"보낸사람:{fr}")
print(f"제목:{subject}")
print(f"내용:{body}")


보낸사람:Fidan Alizada <fidan.alizada@linkgraph.io>
제목:The BEST agency growth secrets of 2023 - SEOTHEORY Q&A 🚀
내용:

View this email in your browser (https://mailchi.mp/4b6d071cf55d/the-best-agency-growth-secrets-of-2023-seotheory-qa?e=50738b472e)

Our weekly SEOTHEORY Q&A Session is coming up!

Last week we had the chance to attend the 7 Figure Agency mastermind...

And in this week's session we're going to be revealing some of the BEST agency growth secrets being implemented this year.

This mastermind was a large investment, but we wanted to share some of the key takeaways with the SEOTHEORY community.

📆 TODAY, November 21, from 1pm-2pm EST
JOIN THE Q&A SESSION HERE (https://us02web.zoom.us/j/82063257201?pwd=VFR1NTZtTkl1MVMrOTBGbVpqaURaZz09)


** WHAT WE'RE GOING TO REVEAL
------------------------------------------------------------
* How to build maximum authority within your niche through monthly webinar funnels, publishing your own niche books, and building testimonial pipelines.
* 